# Prediction on bigger flight (Raster) with New Full Model

In [ ]:
#hide

from osgeo import gdal

from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
pd.options.display.max_rows = 20
pd.options.display.max_columns = 8
import numpy as np
from matplotlib import colors
import rasterio as rio
import rasterio.mask
import fiona
import matplotlib.pyplot as plt
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

def scaleMinMax(x):
    return((x-np.nanmin(x))/(np.nanmax(x)-np.nanmin(x)))

def scaleCCC(x):
    return((x- np.nanpercentile(x,0))/(np.nanpercentile(x,98)-np.nanpercentile(x,0)))



# Set input and Output filename 

In [ ]:
filename = "Belon_1004_MS"
output_name = "Belon_1004_MS_invasea_V3_reframed"

learn = load_learner('../models/DISCOV_Invasea_V3.pkl')
categories = learn.dls.vocab
categories

['Bacillariophyceae', 'Chlorophyta', 'Magnoliopsida', 'Phaeophyta', 'Rhodophyta', 'Sediment', 'SunGlint', 'Water']

In [ ]:
with fiona.open('../Data/shp/mask/' + filename + '.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
with rio.open('../Data/img/' + filename + '.tif') as ds:
    arr, out_transform = rasterio.mask.mask(ds, shapes, crop=True)
    out_meta = ds.meta

gt = [out_transform[2],out_transform[0],out_transform[1],
 out_transform[5],out_transform[3],out_transform[4]]


lowtif = gdal.Open('../Data/img/' + filename + '.tif')
proj = lowtif.GetProjection()
del lowtif 


In [ ]:

Arr_Max=np.max(arr,axis=0)
Arr_Min=np.min(arr,axis=0)

arr_std=(arr - Arr_Min)/(Arr_Max - Arr_Min)


In [ ]:
df_NAN = pd.DataFrame()
df_NAN['Reflectance_444'] = arr[0].ravel()
df_NAN['Reflectance_475'] = arr[1].ravel() 
df_NAN['Reflectance_531'] = arr[2].ravel() 
df_NAN['Reflectance_560'] = arr[3].ravel() 
df_NAN['Reflectance_650'] = arr[4].ravel() 
df_NAN['Reflectance_668'] = arr[5].ravel() 
df_NAN['Reflectance_705'] = arr[6].ravel() 
df_NAN['Reflectance_717'] = arr[7].ravel() 
df_NAN['Reflectance_740'] = arr[8].ravel() 
df_NAN['Reflectance_842'] = arr[9].ravel()

In [ ]:
df_NAN.replace([0,65535], np.nan, inplace=True)

df_NAN['Reflectance_Stan_444'] = arr_std[0].ravel()
df_NAN['Reflectance_Stan_475'] = arr_std[1].ravel() 
df_NAN['Reflectance_Stan_531'] = arr_std[2].ravel() 
df_NAN['Reflectance_Stan_560'] = arr_std[3].ravel() 
df_NAN['Reflectance_Stan_650'] = arr_std[4].ravel() 
df_NAN['Reflectance_Stan_668'] = arr_std[5].ravel() 
df_NAN['Reflectance_Stan_705'] = arr_std[6].ravel() 
df_NAN['Reflectance_Stan_717'] = arr_std[7].ravel() 
df_NAN['Reflectance_Stan_740'] = arr_std[8].ravel() 
df_NAN['Reflectance_Stan_842'] = arr_std[9].ravel()

In [ ]:
df_NAN['NDVI'] = (df_NAN['Reflectance_842']-df_NAN['Reflectance_650'])/(df_NAN['Reflectance_842']+df_NAN['Reflectance_650'])
df_NAN['NDVI_Stan'] = (df_NAN['Reflectance_Stan_842']-df_NAN['Reflectance_Stan_650'])/(df_NAN['Reflectance_Stan_842']+df_NAN['Reflectance_Stan_650'])

In [ ]:
df = df_NAN.dropna()

In [ ]:
df_test_nan_nrum = df_NAN
df_test_nan_nrum['ID'] = np.arange(len(df_test_nan_nrum))
df_test_nrum = df_test_nan_nrum.dropna()

In [ ]:
df.shape

(24779379, 22)

In [ ]:
ID_l=list(df_test_nrum['ID'])

In [ ]:
dl = learn.dls.test_dl(df, bs=4000)
preds,_ = learn.get_preds(dl=dl)

In [ ]:
class_idxs = preds.argmax(axis=1)
# res = [learn.dls.vocab[c] for c in class_idxs]

In [ ]:
class_probs= preds.max(axis=1)

In [ ]:
class_probs=class_probs.values

# Prediction Plot 

In [ ]:
type(class_idxs.tolist())

list

In [ ]:
NumPred= class_idxs.tolist()
PredProbs =class_probs.tolist()

In [ ]:
res_df= pd.DataFrame(list(zip(NumPred, ID_l,PredProbs)),columns =['Pred_ID','ID','Prob'])

In [ ]:
res_df

,Pred_ID,ID,Prob
0,2,70271,0.479743
1,4,76621,0.596987
2,4,76622,0.353299
3,4,82971,0.771637
4,2,82972,0.623290
...,...,...,...
24779374,3,53223154,0.468945
24779375,2,53223155,0.919087
24779376,4,53223156,0.312370
24779377,4,53223157,0.548968


In [ ]:
df_NAN['ID']= np.arange(len(df_NAN))

In [ ]:
res_input_df = pd.merge(df_NAN,res_df, how='left', on = 'ID')

In [ ]:
res_input_df

,Reflectance_444,Reflectance_475,Reflectance_531,Reflectance_560,...,NDVI_Stan,ID,Pred_ID,Prob
0,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN
1,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN
2,NaN,NaN,NaN,NaN,...,NaN,2,NaN,NaN
3,NaN,NaN,NaN,NaN,...,NaN,3,NaN,NaN
4,NaN,NaN,NaN,NaN,...,NaN,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...
53232045,NaN,NaN,NaN,NaN,...,NaN,53232045,NaN,NaN
53232046,NaN,NaN,NaN,NaN,...,NaN,53232046,NaN,NaN
53232047,NaN,NaN,NaN,NaN,...,NaN,53232047,NaN,NaN
53232048,NaN,NaN,NaN,NaN,...,NaN,53232048,NaN,NaN


In [ ]:
Pred_arr = np.asarray(res_input_df['Pred_ID'])

In [ ]:
Pred_arr=Pred_arr+1

In [ ]:
arr.shape

(10, 8383, 6350)

In [ ]:
Prob_arr = np.asarray(res_input_df['Prob'])
Prob_ras = Prob_arr.reshape(arr.shape[1], arr.shape[2])

In [ ]:
Pred_ras = Pred_arr.reshape(arr.shape[1], arr.shape[2])

# Prediction Save

In [ ]:
# export
driver = gdal.GetDriverByName("GTiff")
driver.Register()
outds = driver.Create("../Output/Pred/" + output_name +"_prob.tif", xsize = Prob_ras.shape[1],
                      ysize = Prob_ras.shape[0], bands = 1, 
                      eType = gdal.GDT_Float32)

outds.SetGeoTransform(gt)
outds.SetProjection(proj)
outband = outds.GetRasterBand(1)
outband.WriteArray(Prob_ras)
outband.SetNoDataValue(65535)
outband.FlushCache()

# close your datasets and bands!!!
outband = None
outds = None

driver = gdal.GetDriverByName("GTiff")
driver.Register()
outds = driver.Create("../Output/Pred/" + output_name +"_pred.tif", xsize = Pred_ras.shape[1],
                      ysize = Pred_ras.shape[0], bands = 1, 
                      eType = gdal.GDT_Int16)
outds.SetGeoTransform(gt)
outds.SetProjection(proj)
outband = outds.GetRasterBand(1)
outband.WriteArray(Pred_ras)
outband.SetNoDataValue(65535)
outband.SetNoDataValue(32767)
outband.FlushCache()
# close your datasets and bands!!!
outband = None
outds = None